In [3]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings("default")
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection
import time
from sklearn.neural_network import MLPRegressor

pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000

In [5]:
df_merge = pd.read_excel('DATA/prepared/prepared_hard.xlsx')

In [6]:
ls_steel = df_merge['марка стали'].value_counts().index
len(ls_steel)

63

In [7]:
col_chem = [
     'C', 
    'Mn', 
    'Si', 
    'P', 
    'S',
    'Cr', 
    'Ni', 
    'Cu', 
    'Al', 
    'V', 
    'Ti', 
    'Nb', 
    'Mo', 
    'N', 
    'B'
]

ls = col_chem + ['марка стали']

In [8]:
def mean_chem(df, steel):
    df = df[df['марка стали'] == steel]
    df[col_chem] = df.groupby(['марка стали'])[col_chem].transform(lambda x: round(x.mean(), 5))
    df = df.reset_index(drop=True)
    df = df.drop_duplicates('C')
    df = df[ls]
    return df

In [9]:
df_merge_1 = df_merge[df_merge['марка стали'] == '37Г2Ф']
df_merge_1[col_chem] = df_merge_1.groupby('марка стали')[col_chem].transform(lambda x: round(x.mean(), 5))
df_merge_1 = df_merge_1.reset_index(drop=True)
df_merge_1 = df_merge_1.drop_duplicates('C')
df_merge_1 = df_merge_1[ls]
df_merge_1

D:\Repo\Anaconda2\envs\myenv\lib\site-packages\pandas\core\frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,B,марка стали
0,0.36549,1.51234,0.28287,0.01129,0.00785,0.0951,0.10289,0.1397,0.0288,0.08827,0.00007,0.00011,0.00006,0.0077,NaN,37Г2Ф


In [10]:
df_merge_2 = mean_chem(df_merge, '30Г2')
df_merge_2

D:\Repo\Anaconda2\envs\myenv\lib\site-packages\pandas\core\frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,B,марка стали
0,0.28379,1.33324,0.25709,0.00963,0.00716,0.10085,0.12489,0.19348,0.02663,0.00459,0.00697,0.00569,0.01536,0.00885,0.0005,30Г2


In [11]:
df_tmp = pd.merge(df_merge_1, df_merge_2, how = 'outer')
df_tmp

,C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,B,марка стали
0,0.36549,1.51234,0.28287,0.01129,0.00785,0.09510,0.10289,0.13970,0.02880,0.08827,0.00007,0.00011,0.00006,0.00770,NaN,37Г2Ф
1,0.28379,1.33324,0.25709,0.00963,0.00716,0.10085,0.12489,0.19348,0.02663,0.00459,0.00697,0.00569,0.01536,0.00885,0.0005,30Г2


In [12]:
df_merge_steel = df_tmp
for steel in ls_steel: 
    df_merge_steel =  pd.merge(df_merge_steel, mean_chem(df_merge, steel), how = 'outer')

df_merge_steel

D:\Repo\Anaconda2\envs\myenv\lib\site-packages\pandas\core\frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,B,марка стали
0,0.36549,1.51234,0.28287,0.01129,0.00785,0.09510,0.10289,0.13970,0.02880,0.08827,0.00007,0.00011,0.00006,0.00770,NaN,37Г2Ф
1,0.28379,1.33324,0.25709,0.00963,0.00716,0.10085,0.12489,0.19348,0.02663,0.00459,0.00697,0.00569,0.01536,0.00885,0.00050,30Г2
2,0.14294,0.53463,0.27391,0.00830,0.00404,0.55441,0.08586,0.11095,0.03219,0.05297,0.00169,0.00357,0.00698,0.00620,0.00094,13ХФА
3,0.17652,0.57084,0.27053,0.00847,0.00583,0.99397,0.12437,0.17317,0.03265,0.07316,0.01343,0.04001,0.13938,0.00830,NaN,18ХМФБ
4,0.28418,1.43375,0.26280,0.00889,0.00641,0.09727,0.11977,0.18272,0.02649,0.00555,0.00745,0.00722,0.00998,0.00868,NaN,30Г2-2
5,0.38302,1.57274,0.50352,0.01082,0.00776,0.12014,0.13821,0.18254,0.02756,0.00436,0.00007,0.00002,0.00103,0.00869,0.00100,38Г2С
6,0.29024,1.35896,0.26180,0.00949,0.00654,0.09352,0.11782,0.20908,0.02694,0.00550,0.00900,0.00585,0.00865,0.00901,NaN,30Г2-3
7,0.09229,1.40472,0.59711,0.01108,0.00975,0.08056,0.09859,0.13638,0.02994,0.00595,0.00797,0.00298,0.01152,0.00776,0.00100,09Г2С
8,0.27684,1.35350,0.25814,0.00895,0.00629,0.10788,0.14031,0.19630,0.02886,0.00425,0.00976,0.00540,0.02042,0.01001,NaN,30Г2-8
9,0.32997,0.62696,0.26243,0.01137,0.00996,0.90109,0.08783,0.12683,0.04032,0.00050,0.00087,0.00000,0.00319,0.00433,NaN,32ХА


In [13]:
df_merge_steel.to_excel('mean_chem_steel.xlsx')